In [92]:
def add_col(A,m,i,j):
    n=A.shape[1]
    E=np.eye(n)
    if i==j:
        E[i,i]=m+1
    else:
        E[j,i]=m
    return A@E
def det1(A):
    ZERO=1.0e-100
    n=A.shape[0]
    B=A
    for i in range(n):
        for j in range(n):
            B[i,j]=float(B[i,j])
    d=1
    for i in range(n):
        if abs(B[i,i])<=ZERO:
            x=0
            for col in range(i+1,n):
                if B[i,col]!=0:
                    B=add_col(B,1,i,col)
                    break
                else:
                    x+=1
            if x==n-1-i:
                d=0
                break
            else:
                for row in range(i+1,n):
                    for col in range(n-1,i-1,-1):
                        B[row,col]-=B[row,i]*B[i,col]/B[i,i]
                d=d*B[i,i]
        else:
            for row in range(i+1,n):
                for col in range(n-1,i-1,-1):
                    B[row,col]-=B[row,i]*B[i,col]/B[i,i]
            d=d*B[i,i]
#     d=round(d,5)
    return d

In [79]:
A=np.array([[1.0,0.0,0.0],[0,1,0],[0,0,1]])
print(float(det1(A)))

1.0


In [93]:
def inv(A):
    '''
    Returns the inverse of a NON-SINGULAR SQUARE matrix
    '''
    n=A.shape[0]
    ZERO=1.0e-100
    d=det1(A)
    if abs(d)<=ZERO:
        s="Inverse is NOT possible:This is a singular matrix"
        return s
    A_inv=np.zeros(n*n)
    A_inv=A_inv.reshape(n,n)
    for i in range(n):
        for j in range(n):
            B=np.array([])
            for k in range((n-1)**2):
                B=np.append(B,0.0)
            B=B.reshape(n-1,n-1)
            for row in range(i):
                for col in range(j):
                    B[row,col]=A[row,col]
                for col in range(j+1,n):
                    B[row,col-1]=A[row,col]
            for row in range(i+1,n):
                for col in range(0,j):
                    B[row-1,col]=A[row,col]
                for col in range(j+1,n):
                    B[row-1,col-1]=A[row,col]
            #Now B has become the minor of A[i,j]
            #So cofactor=((-1)**(i+j))*B
            A_inv[i,j]=((-1)**(i+j))*det1(B)/d
#             A_inv[i,j]=round(A_inv[i,j],5)
    A_inv=A_inv. T
    return A_inv

In [1]:
def dot_val(th,X):
    """
    th - (n+1)x1
    X - 1x(n+1)
    """ 
    import numpy as np
    out=X@th
    return out[0][0]
def hval(th,X):
    """
    th - (n+1)x1
    X - 1x(n+1)
    """ 
    from math import exp
#     print(dot_val(th,X))
    out=1/(1+exp((-1)*dot_val(th,X)))
    return out
def sigmoid(x,th):                     
    """
    x - mx(n+1)
    th- (n+1)x1
    return - mx1
    """
    import numpy as np
    from math import exp
    m=x.shape[0]
    out=x@th
    z = (out).astype("float_")   
#     print(z)
    z1=np.zeros(m)
    z1=z1.reshape(m,1)
    for i in range(m):
        z1[i]=1.0/(1.0+exp((-1)*z[i][0]))
    return z1

In [2]:
def log_likelihood(x, y, th):    
    """
    th - (n+1)x1
    x - mx(n+1)
    y- mx1
    return (n+1)x1
    """
    m=y.shape[0]
    import numpy as np
    from math import log
    sigmoid_probs = sigmoid(x, th)   
#     print(sigmoid_probs)
    temp1=np.zeros(m)
    temp2=np.zeros(m)
    temp1=temp1.reshape(m,1)
    temp2=temp2.reshape(m,1)
    for i in range(m):
        temp1[i]=log(sigmoid_probs[i][0])
        temp2[i]=log(1-sigmoid_probs[i][0])
    return (-1)*np.sum(y * temp1
                  + (1 - y) * temp2)/m

In [3]:
def gradient(x, y, th):             
    """
    th - (n+1)x1
    x - mx(n+1)
    y- mx1
    return - (n+1)x1
    """
    m=y.shape[0]
    n=th.shape[0]-1
    import numpy as np
    sigmoid_probs = sigmoid(x, th)        
    #print(sigmoid_probs.shape)
    out=np.sum(((sigmoid_probs-y)*x/m),axis=0)
    out=out.reshape(n+1,1)
    return out

In [19]:
def hessian(x, y,th): 
    """
    th - (n+1)x1
    x - mx(n+1)
    y- mx1
    return - (n+1)x(n+1)
    """
    m=y.shape[0]
    n=th.shape[0]-1
    sigmoid_probs = sigmoid(x,th)                     
#     d1 = np.sum((sigmoid_probs * (1 - sigmoid_probs)) * x * x)                  
#     d2 = np.sum((sigmoid_probs * (1 - sigmoid_probs)) * x * 1)                  
#     d3 = np.sum((sigmoid_probs * (1 - sigmoid_probs)) * 1 * 1)                  
#     H = np.array([[d1, d2],[d2, d3]])   
#     H=np.zeros((n+1)**2)
#     H=H.reshape(n+1,n+1)
#     out=((np.transpose(x))@(sigmoid_probs*(1-sigmoid_probs)*x))
#     for i in range(m):
#         t1=x[i].reshape(1,n+1)
#         t2=np.transpose(t1)
#         ht=sigmoid_probs[i]
#         H=H+(ht*(1-ht))*(t2@t1)
#     H=H/m
    out=((np.transpose(x))@(sigmoid_probs*(1-sigmoid_probs)*x))
    return out/m

In [22]:
def newtons_method(x, y):                                                             
    """
    :param x (np.array(float)): Vector of Boston House Values in dollars
    :param y (np.array(boolean)): Vector of Bools indicting if house has > 2 bedrooms:
    :returns: np.array of logreg's parameters after convergence, [Θ_1, Θ_2]
    th - (n+1)x1
    x - mx(n+1)
    y- mx1
    """
    # Initialize log_likelihood & parameters                                                                   
#     Θ_1 = 15.1                                                                     
#     Θ_2 = -.4 # The intercept term  
    n=x.shape[1]-1
    th=np.zeros(n+1)
    th=th.reshape(n+1,1)
    Δl = np.Infinity                                                                
    l = log_likelihood(x, y,th)      
    print(l)
    # Convergence Conditions                                                        
    δ = .0000000001                                                                 
#     max_iterations = 15                                                            
#     i = 0                                                                           
    while abs(Δl) > δ:                                       
#         i += 1                                                                      
        g = gradient(x, y, th)      
        hess = hessian(x, y, th) 
#         print(hess)
#         print(g)
#         print(g.shape)
        H_inv = np.linalg.pinv(hess)                                                 
        # @ is syntactic sugar for np.dot(H_inv, g.T)¹
#         print(np.linalg.det(hess))
#         print(H_inv)
        d =np.dot(H_inv,g)
#         print(d)
        th=th-d                                                                     
        # Update the log-likelihood at each iteration                                   
        l_new = log_likelihood(x, y,th)                                                      
        Δl = l - l_new 
        print(Δl)
        l = l_new                                                                
    return th

In [6]:
import numpy as np

In [9]:
np.linalg.pinv?

In [10]:
us=[]
for i in range(17):
    us.append(i)
del us[11]
us

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16]

In [11]:
import numpy as np
X=np.loadtxt("../Epoch/Selection_Hackathon/Dataset.csv",delimiter=',',skiprows=321,usecols=us)
X=np.c_[np.ones(1279),X]
for i in range(1279):
    X[i][1]/=10
    X[i][6]/=10
    X[i][7]/=10
    X[i][11]/=10
    X[i][12]/=10
    X[i][15]/=100
#     X[i,1:18]/=100
print(X)
print(X.shape)

[[1.         0.98       0.66       ... 0.793      0.28095238 1.0795    ]
 [1.         0.93       0.61       ... 0.71       0.348      1.117     ]
 [1.         0.78       0.62       ... 0.709      0.3        0.7395    ]
 ...
 [1.         0.63       0.51       ... 0.826      0.13793103 0.754     ]
 [1.         0.59       0.645      ... 0.785      0.1375     0.6615    ]
 [1.         0.6        0.31       ... 0.727      0.23333333 1.2075    ]]
(1279, 17)


In [12]:
te=np.loadtxt("../Epoch/Selection_Hackathon/Dataset.csv",delimiter=',',skiprows=1,max_rows=320,usecols=us)
#print(te.shape)
te=np.c_[np.ones(320),te]
for i in range(320):
    te[i][1]/=10
    te[i][6]/=10
    te[i][7]/=10
    te[i][11]/=10
    te[i][12]/=10
    te[i][15]/=100
#     te[i,1:18]/=100
print(te)
print(te.shape)

[[1.         0.74       0.7        ... 0.636      0.30909091 0.608     ]
 [1.         0.78       0.88       ... 0.778      0.268      0.829     ]
 [1.         0.78       0.76       ... 0.742      0.36       0.744     ]
 ...
 [1.         0.96       0.77       ... 0.722      0.24666667 0.935     ]
 [1.         0.98       0.66       ... 0.793      0.28095238 1.0795    ]
 [1.         0.96       0.77       ... 0.722      0.24666667 0.935     ]]
(320, 17)


In [13]:
teans=np.loadtxt("../Epoch/Selection_Hackathon/Dataset.csv",delimiter=',',skiprows=1,max_rows=320,usecols=[11])
teans=teans.reshape(320,1)
print(teans.shape)

(320, 1)


In [14]:
y=np.loadtxt("../Epoch/Selection_Hackathon/Dataset.csv",delimiter=',',skiprows=321,usecols=[11])
y=y.reshape(1279,1)
print(y)

[[1.]
 [0.]
 [0.]
 ...
 [1.]
 [0.]
 [1.]]


In [23]:
th1=newtons_method(X,y)
th1

0.6931471805599454
0.17648750786890122
0.012297982529911655
0.00043258934908974034
8.841419554261876e-07
7.343459174080635e-12


array([[ 3.31231127e+01],
       [ 2.62897084e-01],
       [-2.70825073e+00],
       [-3.72235788e-01],
       [ 5.77473287e-01],
       [-3.77840338e+00],
       [ 2.23110454e-01],
       [-1.44412066e-01],
       [-3.89538990e+01],
       [-1.29963753e+00],
       [ 3.97708878e+00],
       [ 9.19020308e+00],
       [-7.92804087e-03],
       [-3.96399427e-02],
       [ 1.98685382e-01],
       [ 5.84064085e-01],
       [-1.79040698e+00]])

In [42]:
a=np.array([[1],[2],[3]])
b=np.array([[4],[5],[6]])
(1-a)*b

array([[  0],
       [ -5],
       [-12]])

In [24]:
c=0
for i in range(320):
    temp=te[i].reshape(1,17)
    if (hval(th1,temp)>=0.5 and teans[i]==1.0):
        c+=1
    elif (hval(th1,temp)<0.5 and teans[i]==0.0):
        c+=1
c

223

In [134]:
import numpy as np
x=np.loadtxt("../Epoch/Selection_Hackathon/Dataset.csv",delimiter=',',skiprows=321)
x=np.c_[np.ones(1279),x]
for i in range(1279):
    x[i][1]/=10
    x[i][6]/=10
    x[i][7]/=10
    x[i][11]/=10
    x[i][13]/=10
    x[i][16]/=100
#     X[i,1:18]/=100
print(x)
print(x.shape)
print(x[0])

[[1.         0.98       0.66       ... 0.793      0.28095238 1.0795    ]
 [1.         0.93       0.61       ... 0.71       0.348      1.117     ]
 [1.         0.78       0.62       ... 0.709      0.3        0.7395    ]
 ...
 [1.         0.63       0.51       ... 0.826      0.13793103 0.754     ]
 [1.         0.59       0.645      ... 0.785      0.1375     0.6615    ]
 [1.         0.6        0.31       ... 0.727      0.23333333 1.2075    ]]
(1279, 18)
[1.         0.98       0.66       0.39       3.2        0.083
 2.1        5.9        0.9989     3.37       0.71       1.15
 1.         1.046      5.23       0.793      0.28095238 1.0795    ]


In [135]:
x=np.unique(x,axis=0)

In [136]:
x

array([[1.        , 0.49      , 0.42      , ..., 0.788     , 0.2625    ,
        0.654     ],
       [1.        , 0.5       , 0.38      , ..., 0.798     , 0.23076923,
        0.506     ],
       [1.        , 0.5       , 0.4       , ..., 0.706     , 0.27586207,
        1.413     ],
       ...,
       [1.        , 1.56      , 0.645     , ..., 0.835     , 0.23      ,
        1.4055    ],
       [1.        , 1.56      , 0.685     , ..., 0.78      , 0.71666667,
        1.312     ],
       [1.        , 1.59      , 0.36      , ..., 0.936     , 0.32272727,
        2.428     ]])

In [144]:
y=x[:,12]
y=y.reshape(1081,1)

In [139]:
x1=np.delete(x,12,axis=1)

In [140]:
x1

array([[1.        , 0.49      , 0.42      , ..., 0.788     , 0.2625    ,
        0.654     ],
       [1.        , 0.5       , 0.38      , ..., 0.798     , 0.23076923,
        0.506     ],
       [1.        , 0.5       , 0.4       , ..., 0.706     , 0.27586207,
        1.413     ],
       ...,
       [1.        , 1.56      , 0.645     , ..., 0.835     , 0.23      ,
        1.4055    ],
       [1.        , 1.56      , 0.685     , ..., 0.78      , 0.71666667,
        1.312     ],
       [1.        , 1.59      , 0.36      , ..., 0.936     , 0.32272727,
        2.428     ]])

In [141]:
x1[12]

array([1.        , 0.53      , 0.47      , 0.11      , 2.2       ,
       0.048     , 1.6       , 8.9       , 0.99182   , 3.54      ,
       0.88      , 1.35666667, 0.577     , 2.885     , 0.928     ,
       0.55625   , 0.706     ])

In [145]:
th10=newtons_method(x1,y)

0.6931471805599452
[[-0.06984274]
 [-0.06347364]
 [-0.00681545]
 [-0.03710916]
 [-0.16829325]
 [-0.00379648]
 [-0.07567068]
 [ 0.00247456]
 [-0.06941538]
 [-0.23111933]
 [-0.06814061]
 [-0.09837727]
 [-0.06415518]
 [-0.3207759 ]
 [-0.0719371 ]
 [-0.01183789]
 [-0.05980805]]
[[ 6.57699024e+06 -2.58366281e+06 -2.60408337e+05  3.75964534e+05
   5.67989948e+05  2.54062713e+06 -5.15257467e+02  1.53183073e+02
  -6.70997008e+06  3.30490555e+04  1.59482930e+06 -6.47509450e+04
  -8.78522355e+05  7.05178871e+05 -1.58556464e+06  1.43109500e+03
  -1.88441169e+06]
 [-2.73811653e+06 -7.08015644e+14 -7.08015644e+13 -1.05224267e+14
  -1.57836400e+14 -6.42711786e+14  2.20811130e+02 -6.32436555e+01
   2.79242648e+06 -1.35482265e+04 -3.79651119e+14  2.70849009e+04
   2.68591009e+13  1.36231309e+14  3.79651119e+14 -5.74936598e+02
   5.26121333e+14]
 [-2.75855110e+05 -7.08015644e+13 -7.08015644e+12 -1.05224267e+13
  -1.57836400e+13 -6.42711786e+13  3.17476527e+01 -1.05175083e+01
   2.81499259e+05 -1.438909

In [146]:
th10

array([[-5.1346836 ],
       [ 7.7497735 ],
       [-1.71212739],
       [-1.00448185],
       [-0.98697925],
       [-5.2382946 ],
       [ 0.18621226],
       [-0.13324413],
       [-0.3364334 ],
       [-1.28847073],
       [ 5.6354506 ],
       [ 9.29640825],
       [-7.85218906],
       [-0.06977057],
       [-1.46265078],
       [ 0.43992637],
       [ 3.3022809 ]])

In [151]:
c=0
for i in range(320):
    temp=te[i].reshape(1,17)
    if (hval(th10,temp)>=0.5 and teans[i]==1.0):
        c+=1
    elif (hval(th10,temp)<0.5 and teans[i]==0.0):
        c+=1
c

219